## This notebooks assists with automatically retrieving and processing WSF datasets over a city

In [37]:
import os
import sys
import math
import warnings
import yaml
import pandas as pd
import geopandas as gpd
from osgeo import gdal
import glob
import numpy as np
import rasterio.mask
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
from os.path import exists
from pathlib import Path
from shutil import copyfile
import requests

In [38]:
country = os.getcwd().split('\\')[-1]

In [39]:
city_name = 'Kampala'
AOI_file_name = 'Kampala_AOI_4326.shp'

In [40]:
#aoi_folder = Path('output/AOI')
output_folder = Path('output')
#wsf_types = ['evolution', '2019']
wsf_types = ['evolution']

In [41]:
# cities = pd.read_csv('centroids.csv').city
# centroids = pd.read_csv('centroids.csv')
# epsg_dict = dict(zip(centroids.city, centroids.utm))

In [42]:
def download_wsf(admin_file_name, wsf_type):
    # data_folder = Path(r'E:\World Bank\City Scan') / country / ('data/WSF' + wsf_type)
    data_folder = Path('data') / ('WSF' + wsf_type)
    try:
        os.mkdir(data_folder)
    except FileExistsError:
        pass
    
#     city_no_space = city.replace(" ", "_")
#     city_lower = city_no_space.lower()
#     shp_name = city_no_space + '_AOI.shp'
    shp = gpd.read_file(Path().resolve().parent / 'admin' / admin_file_name)
    
    print('print admin_file crs')
    print(shp.crs)
    
    if shp.crs != 'EPSG:4326':
        print(f'WARNING: CRS is not in 4326')
    
    shp_bounds = shp.bounds
    
    #print(f'length of shp_bounds is: {shp_bounds}')
    
    for i in range(len(shp_bounds)):
        print(f'range is: {i}')
        for x in range(math.floor(shp_bounds.minx[i] - shp_bounds.minx[i] % 2), math.ceil(shp_bounds.maxx[i]), 2):
            for y in range(math.floor(shp_bounds.miny[i] - shp_bounds.miny[i] % 2), math.ceil(shp_bounds.maxy[i]), 2):
                file_name = 'WSF' + wsf_type + '_v1_' + str(x) + '_' + str(y)
                if not exists(data_folder / (file_name + '.tif')):
                    #print(wsf_type)
                    if wsf_type == 'evolution':
                        #print(file_name)
                        file = requests.get('https://download.geoservice.dlr.de/WSF_EVO/files/' + file_name + '/' + file_name + '.tif')
                    elif wsf_type == '2019':
                        file = requests.get('https://download.geoservice.dlr.de/WSF2019/files/' + file_name + '.tif')
                    open(data_folder / (file_name + '.tif'), 'wb').write(file.content)

In [43]:
for wsf_type in wsf_types:
    download_wsf(AOI_file_name, wsf_type)
    #for city in cities:
        #download_wsf(city, wsf_type)

print admin_file crs
EPSG:4326
range is: 0


Mosaic raster by city

In [44]:
def clipdata_wsf(city_name, admin_file_name, wsf_type):
    #data_folder = Path(r'E:\World Bank\City Scan') / country / ('02_urban_change/WSF' + wsf_type)
    data_folder = Path('data') / ('WSF' + wsf_type)
#     city_no_space = city.replace(" ", "_")
#     city_lower = city_no_space.lower()
#     shp_name = city_no_space + '_AOI.shp'
    #shp = gpd.read_file(city / aoi_folder / shp_name)
    shp = gpd.read_file(Path().resolve().parent / 'admin' / admin_file_name)
    
    features = shp.geometry
    
    #input_raster = data_folder / ("WSF" + wsf_type + ".tif")
    # use glob
    #glob.glob('./04_elevation/*.tif')
    input_raster = list(data_folder.glob("*.tif"))[0]
    
    with rasterio.open(input_raster) as src:
        out_image, out_transform = rasterio.mask.mask(
            src, features, crop=True)
        out_meta = src.meta.copy()

        out_meta.update({"driver": "GTiff",
                         "height": out_image.shape[1],
                         "width": out_image.shape[2],
                         "transform": out_transform})

        output_4326_raster_clipped = city_name + "_WSF" + wsf_type + "_4326.tif"

        with rasterio.open(Path().resolve().parent / 'output' / output_4326_raster_clipped, "w", **out_meta) as dest:
            dest.write(out_image)

In [45]:
def find_utm(admin_file_name):
    features_gdf = gpd.read_file(Path().resolve().parent / 'admin' / admin_file_name)
    print('print admin_file crs')
    print(features_gdf.crs)

    # automatically find utm zone
    avg_lng = features_gdf["geometry"].unary_union.centroid.x

    # calculate UTM zone from avg longitude to define CRS to project to
    utm_zone = math.floor((avg_lng + 180) / 6) + 1
    utm_crs = f"+proj=utm +zone={utm_zone} +ellps=WGS84 +datum=WGS84 +units=m +no_defs"
    
    return utm_crs

In [46]:
def utm_wsf(city_name, admin_file_name, wsf_type):
#     city_no_space = city.replace(" ", "_")
#     city_lower = city_no_space.lower()
#     shp_name = city_no_space + '_AOI.shp'
    dst_crs = find_utm(admin_file_name)
    print('print dst_crs')
    print(dst_crs)
    shp = gpd.read_file(Path().resolve().parent / 'admin' / admin_file_name).to_crs(crs = dst_crs)
    features = shp.geometry
    
    with rasterio.open(Path().resolve().parent / 'output' / (city_name + "_WSF" + wsf_type + "_4326.tif")) as src:
        #dst_crs = 'EPSG:' + str(crs)

        transform, width, height = calculate_default_transform(
            src.crs, dst_crs, src.width, src.height, *src.bounds)
        kwargs = src.meta.copy()
        kwargs.update({
            'crs': dst_crs,
            'transform': transform,
            'width': width,
            'height': height
        })

        with rasterio.open(Path().resolve().parent / 'output' / (city_name + '_WSF' + wsf_type + '_utm.tif'), 'w', **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=dst_crs,
                    resampling=Resampling.nearest)
    
    if wsf_type == 'evolution':
        with rasterio.open(Path().resolve().parent / 'output' / (city_name + '_WSF' + wsf_type + '_utm.tif')) as src:
            out_image = src.read()
            pixelSizeX, pixelSizeY = src.res

        year_dict = {}
        for year in range(1985, 2016):
            if year == 1985:
                year_dict[year] = np.count_nonzero(
                out_image == year) * pixelSizeX * pixelSizeY / 1000000
            else:
                year_dict[year] = np.count_nonzero(
                out_image == year) * pixelSizeX * pixelSizeY / 1000000 + year_dict[year-1]

        # save CSV
        with open(Path().resolve().parent / 'output' / (city_name + "_built_up_stats.csv"), 'w') as f:
            f.write("year,cumulative sq km\n")
            for key in year_dict.keys():
                f.write("%s,%s\n" % (key, year_dict[key]))

In [47]:
def reclass_wsf(city, wsf_type):
    city_no_space = city.replace(" ", "_")
    city_lower = city_no_space.lower()
    
    #with rasterio.open(city / output_folder / (city_lower + '_WSF' + wsf_type + '_4326.tif')) as src:
    with rasterio.open(Path().resolve().parent / 'output' / (city_lower + '_WSF' + wsf_type + '_4326.tif')) as src:
        out_image = src.read()
        out_meta = src.meta.copy()
    
    out_image[0][out_image[0] < 1985] = 0
    out_image[0][(out_image[0] <= 2015) & (out_image[0] >= 2006)] = 4
    out_image[0][(out_image[0] < 2006) & (out_image[0] >= 1996)] = 3
    out_image[0][(out_image[0] < 1996) & (out_image[0] >= 1986)] = 2
    out_image[0][out_image[0] == 1985] = 1
    
    out_file = city_lower + '_WSF' + wsf_type + '_reclass.tif'
    with rasterio.open(Path().resolve().parent / 'output' / out_file, "w", **out_meta) as dest:
        dest.write(out_image)

### Post-Process the WSF files and write outputs in the parent's output folder

In [48]:
for wsf_type in wsf_types:
    #for city in cities:
    clipdata_wsf(city_name, AOI_file_name, wsf_type)
    utm_wsf(city_name,AOI_file_name, wsf_type)
    if wsf_type == 'evolution':
        reclass_wsf(city_name, wsf_type)

print admin_file crs
EPSG:4326
print dst_crs
+proj=utm +zone=36 +ellps=WGS84 +datum=WGS84 +units=m +no_defs
